In [1]:

import glob
import json
import os
import numpy as np

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import zscore
from tensorflow import keras
import tensorflow as tf
import pymongo


from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, Activation, Input, Reshape, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.regularizers import l2
from tensorflow.keras import utils

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())



Num GPUs Available:  3
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
Number of devices: 3
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15442637537291333430
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5077532672
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2636407964807696812
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 5077532672
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6850215974410451256
physical_device_desc: "device: 1, name: GeForce GTX 1060 6GB, pci bus id: 0000:03:00.0, compute capability: 6.1"
, name: "/device:GPU:2"
device_type: "GPU"
memory_limit: 5077532672
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13092251282933296554

In [3]:
import gc

def data_to_folders():
    price = []
    size = []
    k = 0
    data = pymongo.MongoClient('localhost',27017)['hitbtc']['btcusd'].find({})
    for c,i in enumerate(data):
        k += 1
        if k%10000 == 0:
            print(k)
        xs = []
        xp = []
        for j in range(20-1,-1,-1):
            xs.append([i['bid'][j]['size']])
            xp.append([i['bid'][j]['price']])
        for j in range(20):
            xs.append([i['ask'][j]['size']])
            xp.append([i['ask'][j]['price']])
        price.append(xp)
        size.append(xs)
    return  price,size
price, size = data_to_folders()

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

In [4]:
price = np.array(price)
size = np.array(size)

In [5]:
import pandas as pd

price = pd.DataFrame(np.squeeze(price))
size = pd.DataFrame(np.squeeze(size))

print(price)
print(size)

               0         1         2         3         4         5         6   \
0        55674.99  55676.35  55678.81  55680.00  55682.64  55685.19  55686.78   
1        55678.81  55680.00  55682.64  55685.19  55686.78  55688.21  55688.30   
2        55674.99  55678.81  55680.00  55682.64  55685.19  55686.78  55688.02   
3        55680.00  55682.64  55685.19  55686.78  55688.02  55688.21  55688.30   
4        55689.17  55689.18  55689.19  55689.84  55690.00  55690.75  55696.40   
...           ...       ...       ...       ...       ...       ...       ...   
2755492  49971.52  49972.45  49977.51  49979.88  49980.00  49980.09  49987.27   
2755493  49971.52  49972.45  49977.51  49979.88  49980.00  49980.09  49980.10   
2755494  49972.45  49977.31  49979.88  49980.00  49980.09  49980.10  49983.93   
2755495  49972.45  49977.31  49979.88  49980.00  49980.09  49980.10  49983.93   
2755496  49972.45  49977.31  49979.88  49980.00  49980.09  49980.10  49983.93   

               7         8 

In [6]:
price['Midprice'] = (price[19] + price[20])/2
size['Midprice'] = (size[19] + size[20])/2

In [7]:
price['sum40'] = price[0]
size['sum40'] = size[0]
for i in range(1,40):
    price['sum40'] += price[i]
    size['sum40'] += size[i]
price['sum40'] /= 40
size['sum40'] /= 40


In [8]:
price['mean'] = price['sum40'].rolling(400000).mean()
size['mean'] = size['sum40'].rolling(400000).mean()

In [9]:
price['sd'] = price['sum40'].rolling(400000).std()
size['sd'] = size['sum40'].rolling(400000).std()

In [10]:
price = price[400000:]
size = size[400000:]

In [11]:
print( (price[0]-price['mean'])/price['sd'] )

400000    -1.998368
400001    -1.995444
400002    -1.995435
400003    -1.995427
400004    -1.993411
             ...   
2755492    0.668972
2755493    0.668971
2755494    0.671098
2755495    0.671097
2755496    0.671096
Length: 2355497, dtype: float64


In [12]:
for i in range(0,40):
    price[i] = price[i]-price['mean']
    size[i] = size[i]-size['mean']

In [13]:
for i in range(0,40):
    price[i] = price[i]/price['sd']
    size[i] = size[i]/size['sd']

In [14]:
print(price)

                0         1         2         3         4         5         6  \
400000  -1.998368 -1.995452 -1.994617 -1.993474 -1.986500 -1.983101 -1.983086   
400001  -1.995444 -1.994609 -1.993436 -1.986492 -1.983093 -1.983078 -1.983063   
400002  -1.995435 -1.994600 -1.993428 -1.986527 -1.983084 -1.983070 -1.983055   
400003  -1.995427 -1.994592 -1.993420 -1.985420 -1.983076 -1.983061 -1.983047   
400004  -1.993411 -1.983053 -1.983038 -1.982980 -1.982042 -1.979932 -1.972782   
...           ...       ...       ...       ...       ...       ...       ...   
2755492  0.668972  0.671100  0.682677  0.688100  0.688375  0.688581  0.705009   
2755493  0.668971  0.671099  0.682677  0.688099  0.688374  0.688580  0.688603   
2755494  0.671098  0.682218  0.688099  0.688373  0.688579  0.688602  0.697365   
2755495  0.671097  0.682217  0.688098  0.688372  0.688578  0.688601  0.697364   
2755496  0.671096  0.682216  0.688097  0.688371  0.688577  0.688600  0.697364   

                7         8

In [15]:
price['avg_mid'] = price['Midprice'].rolling(20).sum()/20

In [16]:
price = price[30:]
size = size[30:]

In [17]:
try:
    for index, row in price.iterrows():
        row['avg_mid'] = (price.loc[index+20,'avg_mid'] - row['avg_mid']) / row['avg_mid']
except:
    pass

In [18]:
price = price[:-40]
size = size[:-40]

In [19]:
goal = price['avg_mid'].to_numpy()
price = price.to_numpy()[:,:40]
size = size.to_numpy()[:,:40]

In [20]:
X = np.zeros((len(price),80))

In [21]:
for i in range(len(price)):
    for j in range(40):
        X[i,2*j] = price[i,j]
        X[i,2*j+1] = size[i,j]

In [85]:
classtrain = []
a = [0,0,0]
for i in goal:
    if i >= 0.0001:
        a[2] += 1
        classtrain.append([0,0,1])
    elif i <-0.0001:
        a[0] +=1
        classtrain.append([1,0,0])
    else:
        a[1] +=1
        classtrain.append([0,1,0])
print(a)

[411238, 1533580, 410609]


In [86]:
classtrain = np.array(classtrain)

In [87]:
print(len(classtrain),len(X))

2355427 2355427


In [88]:
X = np.expand_dims(X,-1)


In [96]:
X_train = tf.keras.preprocessing.sequence.TimeseriesGenerator(
    X[:,20:60,0],
    classtrain,
    100,
    sampling_rate=1,
    stride = 10,
    batch_size=32,
    shuffle=False,
    start_index=0,
    end_index=1100000
)

X_val = tf.keras.preprocessing.sequence.TimeseriesGenerator(
    X[:,20:60,0],
    classtrain,
    100,
    sampling_rate=1,
    stride = 10,
    batch_size=32,
    shuffle=False,
    start_index=1100000,
    
    end_index=1300000
)


X_test = tf.keras.preprocessing.sequence.TimeseriesGenerator(
    X[:,20:60,0],
    classtrain,
    100,
    sampling_rate=1,
    stride = 10,
    batch_size=32,
    shuffle=False,
    start_index=1500000,
    
    end_index=None
)

In [97]:
print(X_train.__getitem__(0)[0].shape)

(32, 100, 40, 1)


In [98]:

# Constants
# batch size for model
BATCH_SIZE = 32

# number of epochs
EPOCHS = 100

# learning rate and epsilon for ADAM optimizer
LEARNING_RATE = 0.01
EPSILON = 1

# threshold to decide which category midprice direction falls in (up, down, stationary)
ALPHA = 0.001


# ## Preprocess the data

# In[ ]:


class Confusion(keras.callbacks.Callback):
    def __init__(self,validation_generator):
        self.modelMIN = None
        self.minloss = 10
        self.validation_data = validation_generator
        self.loss = tf.keras.losses.CategoricalCrossentropy(
            from_logits=False,
            label_smoothing=0,
            reduction="auto",
            name="categorical_crossentropy",
        )
    def on_epoch_end(self, epoch, logs=None):
        y_pred = model.predict_generator(self.validation_data)
        print(y_pred.shape)
        y_test = np.array([self.validation_data.targets[x] for x in range(99,len(self.validation_data.targets[:self.validation_data.end_index]),self.validation_data.stride)])
        loss = self.loss(y_test,y_pred).numpy()
        if loss < self.minloss:
            self.minloss = loss
            self.modelMIN = model
        y_test = np.array([np.argmax(self.validation_data.targets[x]) for x in range(99,len(self.validation_data.targets[:self.validation_data.end_index]),self.validation_data.stride)])
        y_pred = np.array([np.argmax(x) for x in y_pred])
        cm = confusion_matrix(y_test, y_pred)
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm.diagonal()
        acc_each_class = cm.diagonal()

        print('accuracy of each class: \n')
        labels = [0,1,2]
        for i in range(len(labels)):
            print(labels[i], ' : ', acc_each_class[i])
        print('\n')
        print(confusion_matrix(y_test, y_pred, labels=[0,1,2]))
        print('VALIDATION LOSS: ',loss)


mycallback = Confusion(X_val)

In [99]:
def create_deeplob(T, NF, number_of_lstm):
    input_lmd = Input(shape=(T, NF, 1))
    
    # build the convolutional block
    conv_first1 = Conv2D(16, (1, 40), strides=(1, 2))(input_lmd)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
#     conv_first1 = keras.layers.Dropout(0.7)(conv_first1)
#     conv_first1 = Conv2D(16, (4, 1), padding='same')(conv_first1)
#     conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
#     conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
#     conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
#    conv_first1 = keras.layers.Dropout(0.7)(conv_first1)
#    conv_first1 = Conv2D(16, (1, 2), strides=(1, 2))(conv_first1)
#     conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
#     conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
#     conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
#     conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
#    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
#    conv_first1 = keras.layers.Dropout(0.7)(conv_first1)
#    conv_first1 = Conv2D(32, (1, 10))(conv_first1)
#    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
#     conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
#     conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
#     conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
#     conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
#     conv_first1 = keras.layers.Dropout(0.8)(conv_first1)
#     # build the inception module
#     convsecond_1 = Conv2D(32, (1, 1), padding='same')(conv_first1)
#     convsecond_1 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_1)
#     convsecond_1 = Conv2D(64, (3, 1), padding='same')(convsecond_1)
#     convsecond_1 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_1)
#     convsecond_1 = keras.layers.Dropout(0.8)(convsecond_1)
    
#     convsecond_2 = Conv2D(32, (1, 1), padding='same')(conv_first1)
#     convsecond_2 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_2)
#     convsecond_2 = Conv2D(64, (5, 1), padding='same')(convsecond_2)
#     convsecond_2 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_2)
#     convsecond_2 = keras.layers.Dropout(0.8)(convsecond_2)

    
#     convsecond_3 = MaxPooling2D((3, 1), strides=(1, 1), padding='same')(conv_first1)
#     convsecond_3 = Conv2D(64, (1, 1), padding='same')(convsecond_3)
#     convsecond_3 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_3)
#     convsecond_3 = keras.layers.Dropout(0.7)(convsecond_3)

    
    convsecond_output = conv_first1# keras.layers.concatenate([convsecond_1, convsecond_2, convsecond_3], axis=3)

#     # use the MC dropout here
    convsecond_output = keras.layers.Dropout(0.7)(convsecond_output)
    conv_reshape = Reshape((int(convsecond_output.shape[1]), int(convsecond_output.shape[3])))(convsecond_output)
    
    # build the last LSTM layer
    # If GPU is not available, change CuDNNLSTM to LSTM
    conv_lstm = tf.compat.v1.keras.layers.CuDNNLSTM(number_of_lstm,kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0,l2=5e-2))(conv_reshape)
    conv_lstm = keras.layers.Dropout(0.7)(conv_lstm)
# kernel_regularizer=tf.keras.regularizers.l1(0.01),
#                               activity_regularizer=tf.keras.regularizers.l2(0.01)
    #conv_lstm = Dense(64,activation='relu')(conv_lstm)
    # build the output layer
    out = Dense(3, activation='softmax')(conv_lstm)
    model = Model(inputs=input_lmd, outputs=out)
    adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

model = create_deeplob(100, 40, 32)

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 100, 40, 1)]      0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 100, 1, 16)        656       
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 100, 1, 16)        0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 100, 1, 16)        0         
_________________________________________________________________
reshape_7 (Reshape)          (None, 100, 16)           0         
_________________________________________________________________
cu_dnnlstm_7 (CuDNNLSTM)     (None, 32)                6400      
_________________________________________________________________
dropout_29 (Dropout)         (None, 32)                0   

In [112]:

import requests
from sklearn.metrics import recall_score, classification_report
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("best_model_BTC.hdf5", monitor='val_loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

callback = keras.callbacks.EarlyStopping(monitor='val_loss')
print("Batch size:{}, Learning Rate:{}".format(32, 0.01))
#csv_logger =  [mycallback]

history = model.fit_generator(X_train,epochs=300,max_queue_size=500, workers=-1,validation_data=X_val,callbacks=[checkpoint])
score, accuracy = model.evaluate_generator(X_test)
print("Accuracy is {}%".format(accuracy*100))

Batch size:32, Learning Rate:0.01
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 3438 steps, validate for 625 steps
Epoch 1/300
3434/3438 [============================>.] - ETA: 0s - loss: 0.7230 - accuracy: 0.6993
Epoch 00001: val_loss improved from inf to 0.66550, saving model to best_model_BTC.hdf5
3438/3438 [==============================] - 69s 20ms/step - loss: 0.7229 - accuracy: 0.6994 - val_loss: 0.6655 - val_accuracy: 0.7407
Epoch 2/300
3435/3438 [============================>.] - ETA: 0s - loss: 0.7224 - accuracy: 0.7005
Epoch 00002: val_loss did not improve from 0.66550
3438/3438 [==============================] - 64s 19ms/step - loss: 0.7223 - accuracy: 0.7006 - val_loss: 0.6693 - val_accuracy: 0.7494
Epoch 3/300
3435/3438 [============================>.] - ETA: 0s - loss: 0.7214 - accuracy: 0.7007
Epoch 00003: val_loss did not improve from 0.66550
3438/3438 [==============================] - 60s 17ms/step - loss: 0.7216 - accuracy: 0.7006 - val_loss: 0.6925 - 

Epoch 28/300
3437/3438 [============================>.] - ETA: 0s - loss: 0.7203 - accuracy: 0.7020
Epoch 00028: val_loss did not improve from 0.64436
3438/3438 [==============================] - 52s 15ms/step - loss: 0.7202 - accuracy: 0.7020 - val_loss: 0.6603 - val_accuracy: 0.7444
Epoch 29/300
3434/3438 [============================>.] - ETA: 0s - loss: 0.7193 - accuracy: 0.7016
Epoch 00029: val_loss did not improve from 0.64436
3438/3438 [==============================] - 53s 15ms/step - loss: 0.7192 - accuracy: 0.7017 - val_loss: 0.6605 - val_accuracy: 0.7467
Epoch 30/300
3434/3438 [============================>.] - ETA: 0s - loss: 0.7205 - accuracy: 0.7007
Epoch 00030: val_loss did not improve from 0.64436
3438/3438 [==============================] - 52s 15ms/step - loss: 0.7204 - accuracy: 0.7007 - val_loss: 0.6692 - val_accuracy: 0.7504
Epoch 31/300
3434/3438 [============================>.] - ETA: 0s - loss: 0.7201 - accuracy: 0.7026
Epoch 00031: val_loss did not improve from

3437/3438 [============================>.] - ETA: 0s - loss: 0.7226 - accuracy: 0.7005
Epoch 00056: val_loss did not improve from 0.64009
3438/3438 [==============================] - 59s 17ms/step - loss: 0.7227 - accuracy: 0.7005 - val_loss: 0.6715 - val_accuracy: 0.7431
Epoch 57/300
3437/3438 [============================>.] - ETA: 0s - loss: 0.7232 - accuracy: 0.7001
Epoch 00057: val_loss did not improve from 0.64009
3438/3438 [==============================] - 49s 14ms/step - loss: 0.7232 - accuracy: 0.7000 - val_loss: 0.6553 - val_accuracy: 0.7499
Epoch 58/300
3436/3438 [============================>.] - ETA: 0s - loss: 0.7208 - accuracy: 0.7024 ETA: 
Epoch 00058: val_loss did not improve from 0.64009
3438/3438 [==============================] - 48s 14ms/step - loss: 0.7207 - accuracy: 0.7024 - val_loss: 0.6602 - val_accuracy: 0.7464
Epoch 59/300
3434/3438 [============================>.] - ETA: 0s - loss: 0.7268 - accuracy: 0.6980
Epoch 00059: val_loss did not improve from 0.6400

3435/3438 [============================>.] - ETA: 0s - loss: 0.7232 - accuracy: 0.7001
Epoch 00084: val_loss did not improve from 0.64009
3438/3438 [==============================] - 52s 15ms/step - loss: 0.7232 - accuracy: 0.7001 - val_loss: 0.6661 - val_accuracy: 0.7351
Epoch 85/300
3433/3438 [============================>.] - ETA: 0s - loss: 0.7209 - accuracy: 0.7006
Epoch 00085: val_loss did not improve from 0.64009
3438/3438 [==============================] - 59s 17ms/step - loss: 0.7210 - accuracy: 0.7006 - val_loss: 0.6461 - val_accuracy: 0.7512
Epoch 86/300
3437/3438 [============================>.] - ETA: 0s - loss: 0.7270 - accuracy: 0.6996
Epoch 00086: val_loss did not improve from 0.64009
3438/3438 [==============================] - 57s 17ms/step - loss: 0.7269 - accuracy: 0.6996 - val_loss: 0.6623 - val_accuracy: 0.7414
Epoch 87/300
3434/3438 [============================>.] - ETA: 0s - loss: 0.7208 - accuracy: 0.7009
Epoch 00087: val_loss did not improve from 0.64009
3438

Epoch 113/300
3434/3438 [============================>.] - ETA: 0s - loss: 0.7180 - accuracy: 0.7029
Epoch 00113: val_loss did not improve from 0.64009
3438/3438 [==============================] - 59s 17ms/step - loss: 0.7179 - accuracy: 0.7030 - val_loss: 0.6520 - val_accuracy: 0.7516
Epoch 114/300
3433/3438 [============================>.] - ETA: 0s - loss: 0.7184 - accuracy: 0.7025
Epoch 00114: val_loss did not improve from 0.64009
3438/3438 [==============================] - 57s 16ms/step - loss: 0.7183 - accuracy: 0.7026 - val_loss: 0.6410 - val_accuracy: 0.7547
Epoch 115/300
3434/3438 [============================>.] - ETA: 0s - loss: 0.7180 - accuracy: 0.7031
Epoch 00115: val_loss did not improve from 0.64009
3438/3438 [==============================] - 58s 17ms/step - loss: 0.7178 - accuracy: 0.7032 - val_loss: 0.6532 - val_accuracy: 0.7511
Epoch 116/300
3433/3438 [============================>.] - ETA: 0s - loss: 0.7354 - accuracy: 0.6976
Epoch 00116: val_loss did not improve 

...

Epoch 169/300
3436/3438 [============================>.] - ETA: 0s - loss: 0.7192 - accuracy: 0.7015
Epoch 00169: val_loss did not improve from 0.63374
3438/3438 [==============================] - 56s 16ms/step - loss: 0.7193 - accuracy: 0.7014 - val_loss: 0.6456 - val_accuracy: 0.7570
Epoch 170/300
3433/3438 [============================>.] - ETA: 0s - loss: 0.7181 - accuracy: 0.7025 ETA
Epoch 00170: val_loss did not improve from 0.63374
3438/3438 [==============================] - 58s 17ms/step - loss: 0.7180 - accuracy: 0.7025 - val_loss: 0.6446 - val_accuracy: 0.7557
Epoch 171/300
3434/3438 [============================>.] - ETA: 0s - loss: 0.7193 - accuracy: 0.7018
Epoch 00171: val_loss did not improve from 0.63374
3438/3438 [==============================] - 59s 17ms/step - loss: 0.7194 - accuracy: 0.7017 - val_loss: 0.6642 - val_accuracy: 0.7496
Epoch 172/300
3436/3438 [============================>.] - ETA: 0s - loss: 0.7176 - accuracy: 0.7023
Epoch 00172: val_loss did not impr

3436/3438 [============================>.] - ETA: 0s - loss: 0.7160 - accuracy: 0.7031
Epoch 00197: val_loss did not improve from 0.63374
3438/3438 [==============================] - 58s 17ms/step - loss: 0.7161 - accuracy: 0.7030 - val_loss: 0.6594 - val_accuracy: 0.7400
Epoch 198/300
3436/3438 [============================>.] - ETA: 0s - loss: 0.7158 - accuracy: 0.7038
Epoch 00198: val_loss did not improve from 0.63374
3438/3438 [==============================] - 58s 17ms/step - loss: 0.7159 - accuracy: 0.7037 - val_loss: 0.6598 - val_accuracy: 0.7475
Epoch 199/300
3437/3438 [============================>.] - ETA: 0s - loss: 0.7165 - accuracy: 0.7040
Epoch 00199: val_loss did not improve from 0.63374
3438/3438 [==============================] - 58s 17ms/step - loss: 0.7166 - accuracy: 0.7039 - val_loss: 0.6362 - val_accuracy: 0.7550
Epoch 200/300
3436/3438 [============================>.] - ETA: 0s - loss: 0.7188 - accuracy: 0.7023 ETA: 0s - loss: 0.718
Epoch 00200: val_loss did not 

Epoch 225/300
3433/3438 [============================>.] - ETA: 0s - loss: 0.7143 - accuracy: 0.7049
Epoch 00225: val_loss did not improve from 0.63374
3438/3438 [==============================] - 57s 17ms/step - loss: 0.7143 - accuracy: 0.7049 - val_loss: 0.6407 - val_accuracy: 0.7573
Epoch 226/300
3437/3438 [============================>.] - ETA: 0s - loss: 0.7123 - accuracy: 0.7053 ETA: 
Epoch 00226: val_loss did not improve from 0.63374
3438/3438 [==============================] - 58s 17ms/step - loss: 0.7123 - accuracy: 0.7053 - val_loss: 0.6598 - val_accuracy: 0.7501
Epoch 227/300
3436/3438 [============================>.] - ETA: 0s - loss: 0.7140 - accuracy: 0.7048
Epoch 00227: val_loss did not improve from 0.63374
3438/3438 [==============================] - 58s 17ms/step - loss: 0.7139 - accuracy: 0.7049 - val_loss: 0.6418 - val_accuracy: 0.7550
Epoch 228/300
3434/3438 [============================>.] - ETA: 0s - loss: 0.7140 - accuracy: 0.7033 ETA: 1s - l
Epoch 00228: val_los

3437/3438 [============================>.] - ETA: 0s - loss: 0.7138 - accuracy: 0.7041
Epoch 00252: val_loss did not improve from 0.63286
3438/3438 [==============================] - 58s 17ms/step - loss: 0.7139 - accuracy: 0.7041 - val_loss: 0.6483 - val_accuracy: 0.7545
Epoch 253/300
3434/3438 [============================>.] - ETA: 0s - loss: 0.7129 - accuracy: 0.7054
Epoch 00253: val_loss did not improve from 0.63286
3438/3438 [==============================] - 58s 17ms/step - loss: 0.7130 - accuracy: 0.7054 - val_loss: 0.6366 - val_accuracy: 0.7561
Epoch 254/300
3434/3438 [============================>.] - ETA: 0s - loss: 0.7135 - accuracy: 0.7053
Epoch 00254: val_loss did not improve from 0.63286
3438/3438 [==============================] - 57s 17ms/step - loss: 0.7133 - accuracy: 0.7054 - val_loss: 0.6370 - val_accuracy: 0.7553
Epoch 255/300
3433/3438 [============================>.] - ETA: 0s - loss: 0.7137 - accuracy: 0.7051
Epoch 00255: val_loss did not improve from 0.63286
3

3434/3438 [============================>.] - ETA: 0s - loss: 0.7114 - accuracy: 0.7070
Epoch 00280: val_loss did not improve from 0.63203
3438/3438 [==============================] - 58s 17ms/step - loss: 0.7116 - accuracy: 0.7068 - val_loss: 0.6382 - val_accuracy: 0.7517
Epoch 281/300
3436/3438 [============================>.] - ETA: 0s - loss: 0.7126 - accuracy: 0.7047
Epoch 00281: val_loss did not improve from 0.63203
3438/3438 [==============================] - 59s 17ms/step - loss: 0.7126 - accuracy: 0.7047 - val_loss: 0.6388 - val_accuracy: 0.7548
Epoch 282/300
3436/3438 [============================>.] - ETA: 0s - loss: 0.7121 - accuracy: 0.7037
Epoch 00282: val_loss did not improve from 0.63203
3438/3438 [==============================] - 58s 17ms/step - loss: 0.7120 - accuracy: 0.7037 - val_loss: 0.6358 - val_accuracy: 0.7581
Epoch 283/300
3436/3438 [============================>.] - ETA: 0s - loss: 0.7132 - accuracy: 0.7040
Epoch 00283: val_loss did not improve from 0.63203
3

In [113]:
model = tf.keras.models.load_model('best_model_BTC.hdf5')

In [ ]:
pred = model.predict_generator(X_test)

In [ ]:
print(len(pred) )

In [ ]:
prob = [[0,0,0],[0,0,0],[0,0,0]]
for i,val in enumerate(pred):
    act = classtrain[X_test.start_index + i*X_test.stride]
    #print(act)
    a = np.argmax(act)
    p = np.argmax(val)
    #print(a,p)
    prob[a][p] += 1
print(a)

In [ ]:
for i in prob:
    print(i)

In [110]:
add = 0
c = 0
cl = [0,0]
for i,p in enumerate(pred):
    clas = np.argmax(p)
    win = goal[X_test.start_index + i*X_test.stride]
    
    if clas == 0:
        add += -win - 0.0001
        if win < 0:
            cl[0] += 1
        else:
            cl[1] += 1
    elif clas == 2:
        add += win - 0.0001
        if win > 0:
            cl[0] += 1
        else:
            cl[1] += 1
        
print(win)

0.0001740508416779455


In [111]:
print(add,cl)

0.9092586106515577 [9231, 1459]
